# 03. Model Training - Multi-task GreenSpace CNN

This notebook implements and trains the multitask CNN using TensorFlow/Keras:

## Multitask Architecture
- **Backbone**: EfficientNet/ResNet (ImageNet pretrained)
- **Task 1**: Structured rating (1-5 scale) → Regression head
- **Task 2**: Binary features (6 features) → Multi-binary classification
- **Task 3**: Shade level (3 classes) → Categorical classification

## Training Features
- Multitask loss weighting
- Data augmentation
- Learning rate scheduling  
- Early stopping
- Model checkpointing
